In [14]:
from rl import run_inference, ThorNavEnv, CLIPCuriosity
from models import ActorCritic
from cons import DEVICE
import torch
import prior
from ai2thor.controller import Controller

dataset = prior.load_dataset("procthor-10k")
train_scenes = dataset["train"]

NUM_UPDATES = 100

idx = torch.randint(0, len(train_scenes), (1,)).item()
house = train_scenes[idx]

controller = Controller(
    scene=house,
    snapToGrid=False,
    rotateStepDegrees=30,
    renderDepthImage=True,
)



Fetching reference HEAD


[AI2-THOR WARNING] There has been an update to ProcTHOR-10K that must be used with AI2-THOR version 5.0+. To use the new version of ProcTHOR-10K, please update AI2-THOR to version 5.0+ by running:
    pip install --upgrade ai2thor
Alternatively, to downgrade to the old version of ProcTHOR-10K, run:
   prior.load_dataset("procthor-10k", revision="ab3cacd0fc17754d4c080a3fd50b18395fae8647")


Loading test: 100%|██████████| 1000/1000 [00:00<00:00, 24510.32it/s]


In [15]:

# Load trained model
actor_critic = ActorCritic().to(DEVICE)
actor_critic.load_state_dict(torch.load("data/ac_update_30.pt", map_location="cpu"))

clip_curiosity = CLIPCuriosity()
# Disable curiosity during inference
env = ThorNavEnv(controller, clip_curiosity=clip_curiosity)

# Run inference
traj, total = run_inference(actor_critic, env, max_steps=300, deterministic=True)

print("Episode return:", total)

Episode return: -2.0608062614813827


In [11]:
controller.stop()